In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from matplotlib import dates
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.radarmodule as radar
import pyPIPS.polarimetric as dualpol
#from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
import pandas as pd
import xarray as xr
import glob
import numpy.random as random
from scipy.stats import gamma, uniform
from scipy.special import gamma as gammafunc
from scipy import ndimage
from scipy import interpolate
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.calc import wind_components
from metpy.cbook import get_test_data
from metpy.plots import StationPlot
from metpy.plots.wx_symbols import current_weather, sky_cover
from metpy.units import units
from scipy.signal import medfilt2d
import pyart
import cartopy.crs as ccrs
from IPython.display import HTML
%matplotlib inline
# %matplotlib notebook
import warnings;
warnings.filterwarnings('ignore')

In [ ]:
# Read in the gridded radar data
radar_name = 'COW1'

# For 03/30/22 case (PERiLS-2022 IOP2)
date = '0330'
radar_start_datetimestamp = '20220330220355'
radar_end_datetimestamp = '20220331021835'

# Create datetime objects for start and end times
datetime_start = datetime.strptime(radar_start_datetimestamp, '%Y%m%d%H%M%S')
datetime_end = datetime.strptime(radar_end_datetimestamp, '%Y%m%d%H%M%S')

# radar_basedir = \
#     '/Users/dawson29/Projects/PERiLS/obsdata/2022/NEXRAD/GRID/IOP2/KGWX'

radar_basedir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/PERiLS/obsdata/2022/COMMON_GRID/IOP2/COW1'
#radar_basedir = os.path.join(radar_basedir, '{}/{}'.format(date, radar_name[1:]))
gridded_radar_input_dir = radar_basedir
gridded_radar_output_dir = os.path.join(radar_basedir, 'combined')
if not os.path.exists(gridded_radar_output_dir):
    os.makedirs(gridded_radar_output_dir)

radar_start_timestamp = datetime_start.strftime('%H%M%S')
radar_end_timestamp = datetime_end.strftime('%H%M%S')
gridded_radar_combined_filename = '{}_{}_{}_gridded.nc'.format(radar_name, radar_start_timestamp,
                                                               radar_end_timestamp)

gridded_radar_combined_filepath = os.path.join(gridded_radar_output_dir, gridded_radar_combined_filename)

# Read in individual gridded radar volumes, stack them into a combined Dataset and dump to disk
gridded_radar_paths = glob.glob(gridded_radar_input_dir + f'/*.nc')
# print(gridded_radar_paths)

gridded_radar_paths = sorted(gridded_radar_paths)
gridded_radar_input_list = []
#print(gridded_radar_paths)
for path in gridded_radar_paths:
    filename = os.path.basename(path)
    file_timestamp = filename[6:21]
    # print(file_timestamp)
    file_datetime = datetime.strptime(file_timestamp, '%Y%m%d_%H%M%S')
    if file_datetime >= datetime_start and file_datetime <= datetime_end:
        gridded_radar_input_list.append(path)
#print(gridded_radar_input_list)        
gridded_radar_list = []
gridded_radar_xr_list = []
for gridded_radar_path in gridded_radar_input_list:
    print("Reading {}".format(os.path.basename(gridded_radar_path)))
    gridded_radar = pyart.io.read_grid(gridded_radar_path)
    gridded_radar_xr = gridded_radar.to_xarray()
    # For some reason the conversion to xarray does not preserve the latitude, longitude, or altitude of the 
    # grid origin,
    # so add them back here as an attribute. Also, create a new coordinate "alt" that is the height of the 
    # grid origin above sea level
    origin_latitude = gridded_radar.origin_latitude['data'].item()
    origin_longitude = gridded_radar.origin_longitude['data'].item()
    origin_altitude = gridded_radar.origin_altitude['data'].item()
    gridded_radar_xr.attrs['origin_latitude'] = origin_latitude
    gridded_radar_xr.attrs['origin_longitude'] = origin_longitude
    gridded_radar_xr.attrs['origin_altitude'] = origin_altitude
    
    gridded_radar_xr = gridded_radar_xr.assign_coords(alt=gridded_radar_xr.coords['z'] + origin_altitude)
    # print(gridded_radar_xr)

    gridded_radar_xr_list.append(gridded_radar_xr)
gridded_radar_xr = xr.concat(gridded_radar_xr_list, dim='time')
# For some reason, the radar name gets saved incorrectly as an invalid attribute, so fix that here and then write out to disk
gridded_radar_xr.attrs['radar_name'] = radar_name
print("Writing {}".format(gridded_radar_combined_filename))
gridded_radar_xr.to_netcdf(gridded_radar_combined_filepath)

print(gridded_radar_xr)

In [ ]:
# For some reason, the radar name gets saved incorrectly as an invalid attribute, so fix that here and then write out to disk
gridded_radar_xr.attrs['radar_name'] = radar_name
print("Writing {}".format(gridded_radar_combined_filename))
gridded_radar_xr.to_netcdf(gridded_radar_combined_filepath)

In [ ]:
gridded_radar_xr